*Prepared for the course "TDPS22: Data Science Programming" at Jönköping University, Teacher: [Marcel Bollmann](mailto:marcel.bollmann@ju.se)*

# Exercise 7: Data Transformation with dplyr & lubridate

This notebook contains exercises on data transformation. We're following [Chapter 5 in _R for Data Science_](https://r4ds.had.co.nz/transform.html), but also jump ahead a bit and include [Chapter 16 on "dates and times"](https://r4ds.had.co.nz/dates-and-times.html).

Concretely, we'll look at the five core functions for data manipulation with dplyr – `filter()`, `arrange()`, `select()`, `mutate()`, and `summarise()` –, data grouping via `group_by()`, how to use pipes to chain function calls via `%>%`, and how to work with dates, date-times, and times-of-day.

### Learning Goals

- Know how to _perform data manipulation_ with dplyr.
- Understand how to _use pipes_.
- Understand how to _work with dates and date-times_ in lubridate, and _time-of-day_ in hms.

### Useful Resources

+ ["Data transformation" in _R for Data Science_](https://r4ds.had.co.nz/transform.html)
+ ["Dates and times" in _R for Data Science_](https://r4ds.had.co.nz/dates-and-times.html)
+ [RStudio Cheatsheets](https://www.rstudio.com/resources/cheatsheets/)
+ [Hands-On Programming with R](https://rstudio-education.github.io/hopr/) _(as a reference)_

In [1]:
library(tidyverse)
library(lubridate)  # this is part of Tidyverse, but needs to be loaded explicitly
library(hms)        # this is part of Tidyverse, but needs to be loaded explicitly

Warning message:
“replacing previous import ‘lifecycle::last_warnings’ by ‘rlang::last_warnings’ when loading ‘tibble’”
Warning message:
“replacing previous import ‘lifecycle::last_warnings’ by ‘rlang::last_warnings’ when loading ‘hms’”
── Attaching packages ────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────── tidyverse 1.3.1 ──

✔ ggplot2 3.3.5     ✔ purrr   0.3.4
✔ tibble  3.1.4     ✔ dplyr   1.0.7
✔ tidyr   1.1.3     ✔ stringr 1.4.0
✔ readr   2.0.1     ✔ forcats 0.5.1

── Conflicts ───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()


Attaching package: ‘lubridate’


The following objects are masked from ‘package:base’:

    date, intersect, setdi

- - - 

Let's try loading the **Coffee Chain dataset** that we already saw in Exercise 1:

In [2]:
coffee <- read_csv("data/coffee-chain.csv")

Rows: 10000 Columns: 7
── Column specification ───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
Delimiter: ","
chr (4): Ddate, Market, Product, Product Type
dbl (3): Sales, Profit, Expenses

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


The column types are determined automatically, but they are not ideal — the numeric columns are assigned `dbl` (floating-point numbers) while they are actually only ever integer-valued, and the "Market", "Product", "Product Type" columns are good examples for **factors**, since they are categorical variables with a fixed set of possible values. Of course, there's also the question of how to parse the date into a proper `date` variable, but we'll get to that a little later. The other issues are easy to fix, so let's do that immediately:

In [3]:
coffee <- read_csv("data/coffee-chain.csv", col_types="cfffiii")
str(coffee)

spec_tbl_df [10,000 × 7] (S3: spec_tbl_df/tbl_df/tbl/data.frame)
 $ Ddate       : chr [1:10000] "9/1/13" "5/1/12" "1/1/13" "8/1/12" ...
 $ Market      : Factor w/ 4 levels "Central","East",..: 1 2 3 3 3 2 1 1 4 1 ...
 $ Product     : Factor w/ 13 levels "Decaf Irish Cream",..: 1 2 3 4 5 6 7 1 2 1 ...
 $ Product Type: Factor w/ 4 levels "Coffee","Espresso",..: 1 2 2 3 1 4 2 1 2 1 ...
 $ Sales       : int [1:10000] 90 203 524 273 201 171 132 225 108 208 ...
 $ Profit      : int [1:10000] 36 56 136 81 -3 26 72 160 37 105 ...
 $ Expenses    : int [1:10000] 37 55 93 68 68 73 31 56 42 43 ...
 - attr(*, "spec")=
  .. cols(
  ..   Ddate = col_character(),
  ..   Market = col_factor(levels = NULL, ordered = FALSE, include_na = FALSE),
  ..   Product = col_factor(levels = NULL, ordered = FALSE, include_na = FALSE),
  ..   `Product Type` = col_factor(levels = NULL, ordered = FALSE, include_na = FALSE),
  ..   Sales = col_integer(),
  ..   Profit = col_integer(),
  ..   Expenses = col_integer()
  

Look at the help for `read_csv()` if you want to know more about the `col_types` argument ...

- - - 

## Filtering, arranging, selecting, mutating

We first look at using four core dplyr functions individually:
- `filter()` is a way to select certain _rows_ of a dataset;
- `arrange()` is a way to sort them;
- `select()` is a way to select certain _columns_ of a dataset; and
- `mutate()` is a way to create new columns from old ones.

**1. Select all rows with products of type "Coffee"!**  _(You should end up with 3383 rows. Look at the first line of the output to see the number of rows!)_

In [4]:
filter(coffee, `Product Type` == "Coffee")

Ddate,Market,Product,Product Type,Sales,Profit,Expenses
<chr>,<fct>,<fct>,<fct>,<int>,<int>,<int>
9/1/13,Central,Decaf Irish Cream,Coffee,90,36,37
12/1/13,West,Amaretto,Coffee,201,-3,68
2/1/13,Central,Decaf Irish Cream,Coffee,225,160,56
9/1/13,Central,Decaf Irish Cream,Coffee,208,105,43
1/1/12,Central,Decaf Irish Cream,Coffee,316,163,64
1/1/12,South,Decaf Irish Cream,Coffee,112,66,24
2/1/13,South,Decaf Irish Cream,Coffee,130,68,43
1/1/12,East,Decaf Irish Cream,Coffee,252,87,81
7/1/13,Central,Decaf Irish Cream,Coffee,70,24,22


**2. Select all rows with negative profit values!** _(You should end up with 686 rows.)_

In [5]:
filter(coffee, Profit < 0)

Ddate,Market,Product,Product Type,Sales,Profit,Expenses
<chr>,<fct>,<fct>,<fct>,<int>,<int>,<int>
12/1/13,West,Amaretto,Coffee,201,-3,68
9/1/12,East,Lemon,Herbal Tea,84,-8,72
4/1/12,East,Decaf Espresso,Espresso,54,-4,26
9/1/12,South,Lemon,Herbal Tea,145,-4,54
9/1/12,South,Lemon,Herbal Tea,89,-5,37
7/1/12,West,Decaf Irish Cream,Coffee,127,-2,71
5/1/13,East,Decaf Irish Cream,Coffee,185,-56,49
5/1/13,West,Mint,Herbal Tea,109,-7,71
9/1/12,South,Decaf Irish Cream,Coffee,77,-16,72


**3. Select all rows with product type "Coffee" _or_ "Espresso" that also come from the "West" market!** _(You should end up with 1712 rows.)_

In [6]:
filter(coffee, Market == "West", `Product Type` %in% c("Coffee", "Espresso"))

Ddate,Market,Product,Product Type,Sales,Profit,Expenses
<chr>,<fct>,<fct>,<fct>,<int>,<int>,<int>
1/1/13,West,Caffe Latte,Espresso,524,136,93
12/1/13,West,Amaretto,Coffee,201,-3,68
10/1/12,West,Decaf Irish Cream,Coffee,114,16,51
2/1/12,West,Decaf Irish Cream,Coffee,109,6,55
10/1/12,West,Decaf Irish Cream,Coffee,134,33,58
7/1/12,West,Decaf Irish Cream,Coffee,148,43,64
2/1/12,West,Decaf Espresso,Espresso,105,30,25
12/1/12,West,Caffe Mocha,Espresso,200,53,52
5/1/13,West,Decaf Irish Cream,Coffee,164,46,48


**4. Select all rows with "Darjeeling" tea where the _sales_ are greater than 500!** _(You should end up with 76 rows.)_

In [7]:
filter(coffee, Product == "Darjeeling", Sales > 500)

Ddate,Market,Product,Product Type,Sales,Profit,Expenses
<chr>,<fct>,<fct>,<fct>,<int>,<int>,<int>
10/1/13,Central,Darjeeling,Tea,546,157,121
10/1/13,Central,Darjeeling,Tea,543,132,89
6/1/13,West,Darjeeling,Tea,537,220,104
10/1/13,West,Darjeeling,Tea,510,153,93
10/1/13,West,Darjeeling,Tea,509,210,120
7/1/13,West,Darjeeling,Tea,593,238,123
6/1/13,Central,Darjeeling,Tea,585,148,115
10/1/13,Central,Darjeeling,Tea,516,133,115
10/1/13,West,Darjeeling,Tea,610,224,129


**5. Arrange the coffee dataset by "Sales" in descending order!** What product had the highest number of sales in the dataset?

In [8]:
arrange(coffee, desc(Sales))

Ddate,Market,Product,Product Type,Sales,Profit,Expenses
<chr>,<fct>,<fct>,<fct>,<int>,<int>,<int>
8/1/12,West,Lemon,Herbal Tea,902,228,178
8/1/13,East,Lemon,Herbal Tea,806,368,95
8/1/13,West,Chamomile,Herbal Tea,791,432,137
3/1/13,East,Decaf Irish Cream,Coffee,781,276,159
7/1/12,East,Decaf Irish Cream,Coffee,754,256,188
7/1/12,West,Colombian,Coffee,750,190,160
8/1/12,West,Lemon,Herbal Tea,737,234,156
8/1/13,Central,Colombian,Coffee,726,317,100
3/1/13,East,Decaf Irish Cream,Coffee,723,476,138


**6. Select the "Product" and "Product Type" columns and assign them to a variable `products`!**

In [9]:
products <- select(coffee, contains("Product"))

Afterwards, you should be able to run the following line of code to get all 13 unique "Product"–"Product Type" combinations:

In [10]:
unique(products)

Product,Product Type
<fct>,<fct>
Decaf Irish Cream,Coffee
Decaf Espresso,Espresso
Caffe Latte,Espresso
Earl Grey,Tea
Amaretto,Coffee
Lemon,Herbal Tea
Caffe Mocha,Espresso
Mint,Herbal Tea
Darjeeling,Tea


**7. Find out what the `any_of()` function does, and how to use it to select all columns whose names are included in the `cols` vector below.**

In [11]:
cols <- c("Sales", "Inventory", "Budget", "Profit", "Expenses")

In [12]:
select(coffee, any_of(cols))

Sales,Profit,Expenses
<int>,<int>,<int>
90,36,37
203,56,55
524,136,93
273,81,68
201,-3,68
171,26,73
132,72,31
225,160,56
108,37,42


**8. Rearrange the columns so that the "Product Type" column comes first.**

In [13]:
select(coffee, `Product Type`, everything())

Product Type,Ddate,Market,Product,Sales,Profit,Expenses
<fct>,<chr>,<fct>,<fct>,<int>,<int>,<int>
Coffee,9/1/13,Central,Decaf Irish Cream,90,36,37
Espresso,5/1/12,East,Decaf Espresso,203,56,55
Espresso,1/1/13,West,Caffe Latte,524,136,93
Tea,8/1/12,West,Earl Grey,273,81,68
Coffee,12/1/13,West,Amaretto,201,-3,68
Herbal Tea,9/1/12,East,Lemon,171,26,73
Espresso,9/1/13,Central,Caffe Mocha,132,72,31
Coffee,2/1/13,Central,Decaf Irish Cream,225,160,56
Espresso,10/1/12,South,Decaf Espresso,108,37,42


**9. Rename the "Product Type" column to "Product_Type"!** Notice the underscore. If we change the space to an underscore, we don't have to wrap this column name in backticks all the time. Assign the result to the `coffee` variable again so the change persists!

In [14]:
coffee <- rename(coffee, Product_Type = `Product Type`)

_Bonus:_ The `rename_with()` function changes column names based on a function. For example, we can change them all to lowercase via:

In [15]:
coffee <- rename_with(coffee, tolower)

I'll refer to column names in lowercase from here on.

**10. Create two new columns: a column "revenue" that is the _sum_ of "profit" and "expenses"; and a column "margin" that is "profit" _divided by_ "revenue".** Try to create both columns in a single `mutate()` statement.

In [16]:
(coffee <- mutate(coffee, revenue = profit + expenses, margin = profit / revenue))

ddate,market,product,product_type,sales,profit,expenses,revenue,margin
<chr>,<fct>,<fct>,<fct>,<int>,<int>,<int>,<int>,<dbl>
9/1/13,Central,Decaf Irish Cream,Coffee,90,36,37,73,0.49315068
5/1/12,East,Decaf Espresso,Espresso,203,56,55,111,0.50450450
1/1/13,West,Caffe Latte,Espresso,524,136,93,229,0.59388646
8/1/12,West,Earl Grey,Tea,273,81,68,149,0.54362416
12/1/13,West,Amaretto,Coffee,201,-3,68,65,-0.04615385
9/1/12,East,Lemon,Herbal Tea,171,26,73,99,0.26262626
9/1/13,Central,Caffe Mocha,Espresso,132,72,31,103,0.69902913
2/1/13,Central,Decaf Irish Cream,Coffee,225,160,56,216,0.74074074
10/1/12,South,Decaf Espresso,Espresso,108,37,42,79,0.46835443


**11. Make a new column "profit_above_avg" that is `TRUE` when the "profit" is above the average of the dataset, and `FALSE` otherwise.** _Note:_ You can get the average by calling `mean()`.

In [17]:
(coffee <- mutate(coffee, profit_above_avg = profit > mean(profit)))

ddate,market,product,product_type,sales,profit,expenses,revenue,margin,profit_above_avg
<chr>,<fct>,<fct>,<fct>,<int>,<int>,<int>,<int>,<dbl>,<lgl>
9/1/13,Central,Decaf Irish Cream,Coffee,90,36,37,73,0.49315068,FALSE
5/1/12,East,Decaf Espresso,Espresso,203,56,55,111,0.50450450,FALSE
1/1/13,West,Caffe Latte,Espresso,524,136,93,229,0.59388646,TRUE
8/1/12,West,Earl Grey,Tea,273,81,68,149,0.54362416,TRUE
12/1/13,West,Amaretto,Coffee,201,-3,68,65,-0.04615385,FALSE
9/1/12,East,Lemon,Herbal Tea,171,26,73,99,0.26262626,FALSE
9/1/13,Central,Caffe Mocha,Espresso,132,72,31,103,0.69902913,TRUE
2/1/13,Central,Decaf Irish Cream,Coffee,225,160,56,216,0.74074074,TRUE
10/1/12,South,Decaf Espresso,Espresso,108,37,42,79,0.46835443,FALSE


- - - 

## Pipes, grouping, and summarising

_Grouping_ and _summarising_ data is mainly done via `group_by()` and `summarise()`. They are often used together, and as such, it's a good idea to take a look at _pipes_ first before we get more into them.

**12. Rewrite the cell below to a version _without pipes_ and _only one function per line_.** Use intermediate variables to store results of function calls. Basically, make sure that you understand what exactly is happening in the pipe.

In [18]:
coffee %>%  
  pull(profit) %>%
  sum == 643034

[1] TRUE

In [19]:
tmp <- pull(coffee, profit)
tmp <- sum(tmp)
tmp == 643034

[1] TRUE

**13. Rewrite the cell below to a version _with pipes_.** You can use the `pull()` function to "pull out" specific columns of the dataset, or use the dot `.` in a pipe as a placeholder for the input variable.

In [20]:
tmp <- filter(coffee, sales > 100)
tmp <- count(tmp)
tmp$n

[1] 7444

In [21]:
coffee %>% filter(sales > 100) %>% count %>% pull(n)
# or
coffee %>% filter(sales > 100) %>% count %>% .$n

[1] 7444

[1] 7444

**14. How many instances of each product type are there, and which one generates the most/the least profit, on average?** You'll need to chain a `group_by()` and a `summarise()` call to solve this; try to do this with a pipe (`%>%`) instead of intermediate variables, if possible. You should end up with a tibble that has four rows, one for each product type, and columns for the _name_, _count_, and _mean profit_ of each product type.

In [22]:
coffee %>%
  group_by(product_type) %>%
  summarise(
      count = n(),
      mean_profit = mean(profit),
  )

product_type,count,mean_profit
<fct>,<int>,<dbl>
Coffee,3383,75.57819
Espresso,2408,58.97301
Tea,1714,51.12602
Herbal Tea,2495,63.21283


**15. What's the _minimum, maximum, average, and median profit_ per "product"?** The resulting tibble should have unique "product" values (e.g., Amaretto, Caffe Latte, ...) as _rows_, and the statistics about the distribution of their "profit" values as _columns_.

In [23]:
coffee %>%
  group_by(product) %>%
  summarise(
      count = n(),
      min(profit),
      mean(profit),
      max(profit),
      median(profit)
  )

product,count,min(profit),mean(profit),max(profit),median(profit)
<fct>,<int>,<int>,<dbl>,<int>,<dbl>
Decaf Irish Cream,3116,-366,71.02246,778,42.0
Decaf Espresso,1149,-301,59.00957,329,41.0
Caffe Latte,292,-27,46.11644,315,31.0
Earl Grey,79,-30,89.03797,234,86.0
Amaretto,166,-40,57.25301,371,33.0
Lemon,1143,-214,63.99825,448,37.0
Caffe Mocha,919,-321,64.00109,437,41.0
Mint,354,-289,53.99153,277,42.0
Darjeeling,1626,-586,49.45941,283,33.0


**16. Filter out all products where we don't have data from all markets.** This is potentially a bit trickier. You might want to start by finding an expression that gives you the number of unique markets (you've seen an example for getting unique values at the top of this notebook, and you can look at the `length()` function to obtain a count), then try to figure out how to use grouping and filtering to achieve the desired result.

_Note:_ Like in Exercise 1, where we did this before in Python, an indication that you got the right expression is that you should end up with a filtered dataset that has 7426 rows.

In [24]:
coffee %>%
  group_by(product) %>%
  filter(length(unique(market)) == 4)   # alternatively: n_distinct(market) == 4

ddate,market,product,product_type,sales,profit,expenses,revenue,margin,profit_above_avg
<chr>,<fct>,<fct>,<fct>,<int>,<int>,<int>,<int>,<dbl>,<lgl>
9/1/13,Central,Decaf Irish Cream,Coffee,90,36,37,73,0.49315068,FALSE
5/1/12,East,Decaf Espresso,Espresso,203,56,55,111,0.50450450,FALSE
9/1/12,East,Lemon,Herbal Tea,171,26,73,99,0.26262626,FALSE
9/1/13,Central,Caffe Mocha,Espresso,132,72,31,103,0.69902913,TRUE
2/1/13,Central,Decaf Irish Cream,Coffee,225,160,56,216,0.74074074,TRUE
10/1/12,South,Decaf Espresso,Espresso,108,37,42,79,0.46835443,FALSE
9/1/13,Central,Decaf Irish Cream,Coffee,208,105,43,148,0.70945946,TRUE
5/1/12,East,Lemon,Herbal Tea,214,48,59,107,0.44859813,FALSE
1/1/12,Central,Decaf Irish Cream,Coffee,316,163,64,227,0.71806167,TRUE


- - - 

## Dates and date-times

In this final part, we combine what we've practised so far with `lubridate`, which gives us functionality to work with dates and date-times. Here, [Chapter 16](https://r4ds.had.co.nz/dates-and-times.html) of the book comes into play.

**17. Find the right `lubridate` function to convert the following string to a `date` object!** This string is in the same format as the "Ddate" column of our coffee dataset, so knowing the conversion function will help us work with it.

In [25]:
ddate1 <- "8/21/13"  # a.k.a. 21st August, 2013

In [26]:
mdy(ddate1)  # mdy = month - day - year.
# Note that we don't have to specify if these are separated by slashes or hyphens or something else; lubridate figures that out for us.

[1] "2013-08-21"

**18. Modify the `coffee` dataset so that the "ddate" column contains a parsed "date" object instead of a string!** In the transformed dataset, "ddate" should now be a column of type `<date>` instead of `<chr>`.

In [27]:
(coffee <- mutate(coffee, ddate = mdy(ddate)))

ddate,market,product,product_type,sales,profit,expenses,revenue,margin,profit_above_avg
<date>,<fct>,<fct>,<fct>,<int>,<int>,<int>,<int>,<dbl>,<lgl>
2013-09-01,Central,Decaf Irish Cream,Coffee,90,36,37,73,0.49315068,FALSE
2012-05-01,East,Decaf Espresso,Espresso,203,56,55,111,0.50450450,FALSE
2013-01-01,West,Caffe Latte,Espresso,524,136,93,229,0.59388646,TRUE
2012-08-01,West,Earl Grey,Tea,273,81,68,149,0.54362416,TRUE
2013-12-01,West,Amaretto,Coffee,201,-3,68,65,-0.04615385,FALSE
2012-09-01,East,Lemon,Herbal Tea,171,26,73,99,0.26262626,FALSE
2013-09-01,Central,Caffe Mocha,Espresso,132,72,31,103,0.69902913,TRUE
2013-02-01,Central,Decaf Irish Cream,Coffee,225,160,56,216,0.74074074,TRUE
2012-10-01,South,Decaf Espresso,Espresso,108,37,42,79,0.46835443,FALSE


**19. Select all _rows_ of the dataset where the _month_ is August.** Use your transformed dataset from the previous question.

In [28]:
filter(coffee, month(ddate) == 8)

ddate,market,product,product_type,sales,profit,expenses,revenue,margin,profit_above_avg
<date>,<fct>,<fct>,<fct>,<int>,<int>,<int>,<int>,<dbl>,<lgl>
2012-08-01,West,Earl Grey,Tea,273,81,68,149,0.5436242,TRUE
2012-08-01,South,Caffe Mocha,Espresso,49,23,17,40,0.5750000,FALSE
2012-08-01,West,Darjeeling,Tea,345,122,46,168,0.7261905,TRUE
2013-08-01,South,Lemon,Herbal Tea,84,32,19,51,0.6274510,FALSE
2012-08-01,South,Decaf Espresso,Espresso,78,0,34,34,0.0000000,FALSE
2013-08-01,Central,Decaf Irish Cream,Coffee,350,214,105,319,0.6708464,TRUE
2012-08-01,South,Caffe Latte,Espresso,48,10,15,25,0.4000000,FALSE
2012-08-01,South,Lemon,Herbal Tea,82,8,22,30,0.2666667,FALSE
2012-08-01,South,Caffe Mocha,Espresso,311,128,71,199,0.6432161,TRUE


**20. How many "ddate"s in the dataset fell on a Sunday, and how many rows do we have for these dates?** I'll give you the answer in table form below – produce a function pipeline that ultimately gives the same result:

|   ddate    | count | wday   |
|------------|-------|--------|
| 2012-01-01 | 565   | Sunday |
| 2012-04-01 | 434   | Sunday |
| 2012-07-01 | 431   | Sunday |
| 2013-09-01 | 410   | Sunday |
| 2013-12-01 | 99    | Sunday |

In [29]:
coffee %>%
  group_by(ddate) %>%
  summarise(
      count = n(),
      wday = wday(unique(ddate), label = TRUE, abbr = FALSE),
  ) %>%
  filter(
      wday == "Sunday"
  )

ddate,count,wday
<date>,<int>,<ord>
2012-01-01,565,Sunday
2012-04-01,434,Sunday
2012-07-01,431,Sunday
2013-09-01,410,Sunday
2013-12-01,99,Sunday


The "coffee chain" dataset doesn't contain any times or date-times, so we'll try to answer some "artificial" questions that deal with those for now. There will be more opportunities to practice working with times and date-times in the assignment (and potentially other exercises).

**21. How many _seconds_ passed between the birth of _Queen Elizabeth II of the United Kingdom_ and _King Carl XVI Gustaf of Sweden_?** Wikipedia is very specific about the times that royals were born. Concretely, you can find that:

- Queen Elizabeth II was born on April 21, 1926, at 2:40 a.m. GMT in London.
- Carl XVI Gustaf was born on April 30, 1946, at 10:20 a.m. in Stockholm.

Create date-time objects for the birth of these two monarchs (don't forget about time zones!) and compute the difference _in seconds_ between them!

In [30]:
(elizabeth <- dmy_hm("21-04-1926 02:40", tz="GMT"))
(carlgustaf <- dmy_hm("30-04-1946 10:20", tz="Europe/Stockholm"))
as.duration(carlgustaf - elizabeth)

[1] "1926-04-21 02:40:00 GMT"

[1] "1946-04-30 10:20:00 CET"

[1] "631953600s (~20.03 years)"

**22. How many seconds after 10:00:00 is it right now?** This is a use case for the `hms` package, which isn't explicitly described in the book, but is simple & useful enough that we can just take a brief look at its documentation. Check `?hms` or [the "hms" website](https://hms.tidyverse.org/) for a brief overview of the package, and use the `hms()`, `as_hms()`, and `now()` functions to answer the question.

In [31]:
as_hms(now()) - hms(0, 0, 10)

Time difference of 7903.683 secs